In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import ast
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import inflect
import seaborn as sns
import spacy
import json

from models import Preprocessing
from models import IngredientMatcher

In [2]:
df_RAW_recipes = pd.read_csv('/Users/ramzi/Projet_Kit_Big_Data/data/RAW_recipes.csv')
df_pp_recipes=pd.read_csv('/Users/ramzi/Projet_Kit_Big_Data/data/PP_recipes.csv')

# Nettoyer les données
df_RAW_recipes.submitted=pd.to_datetime(df_RAW_recipes.submitted)
df_RAW_recipes_merged=df_RAW_recipes.copy()
df_RAW_recipes_merged=df_RAW_recipes_merged.merge(df_pp_recipes,how='inner',on='id')
df_RAW_recipes_merged[df_RAW_recipes_merged.isna().any(axis=1)] # Verification présence de NaN
df_RAW_recipes_merged.ingredients =  df_RAW_recipes_merged.ingredients.apply(lambda x : ast.literal_eval(x) )

# Instanciation de la classe Preprocessing
preprocessor = Preprocessing(df_RAW_recipes_merged)


/Users/ramzi/Library/Caches/pypoetry/virtualenvs/etude-app-cuisine-pmmUL2vP-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ramzi/Library/Caches/pypoetry/virtualenvs/etude-app-cuisine-pmmUL2vP-py3.10/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/ramzi/Library/Caches/pypoetry/virtualenvs/etude-app-cuisine-pmmUL2vP-py3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
mapping = preprocessor.mapping()

In [4]:
preprocessor.tokenised_recipe(df_RAW_recipes_merged.ingredients.iloc[0], mapping)

['squash', 'seasoning', 'spice', 'honey', 'butter', 'oil', 'salt']

In [5]:
dico_all_month_ingredient = {}

# Traitement pour chaque mois
for month in range(1, 13):
    # Création du dictionnaire pour chaque mois
    monthly_dico = preprocessor.create_monthly_dico_ingredient(month)
    counted_ingredients = preprocessor.count_ingredients(monthly_dico)
    lemmatized_and_sorted = preprocessor.lemmetize_and_sort(counted_ingredients)
    dico_all_month_ingredient[month] = lemmatized_and_sorted

In [6]:
df_recipes_tokenised=df_RAW_recipes_merged.copy()
df_recipes_tokenised.ingredients=df_recipes_tokenised.ingredients.apply(lambda x: preprocessor.tokenised_recipe(x,mapping) )

In [7]:
matcher = IngredientMatcher(df_recipes_tokenised,dico_all_month_ingredient)

In [8]:
dico_all_month_ingredient[1].loc['strawberry'].freq

np.float64(1.2096056922620815)

In [9]:
matcher.ingredient_best_seasonal('strawberry')

5

In [10]:
matcher.ingredient_std('strawberry')

(array([[0.34681535],
        [0.41106478],
        [0.44569378],
        [0.62530427],
        [1.        ],
        [0.65598699],
        [0.60430898],
        [0.42625631],
        [0.27107403],
        [0.20268351],
        [0.2087041 ],
        [0.25033558]]),
 [np.float64(0.22421985780472514), 12])

In [11]:
id_top_receipes = matcher.seasonal_recommendations_1('strawberry',2)


In [15]:
# etapes pour sauvegarder df_tokenised_recipes avec les dates au bon format 

df_recipes_tokenised_test = df_recipes_tokenised.copy()
df_recipes_tokenised_test['submitted'] = df_recipes_tokenised_test['submitted'].dt.strftime('%Y-%m-%d')

#Avant d'executer mettre en cache cette ligne etude_app_cuisine/src/data
#df_recipes_tokenised_test.to_json("etude_app_cuisine/src/data/data_recipes_tokenised.json", orient="records", indent=4)


In [17]:
df_loaded = pd.read_json("etude_app_cuisine/src/data/data_recipes_tokenised.json", orient="records")
df_loaded['submitted'] = pd.to_datetime(df_loaded['submitted'])


FileNotFoundError: File etude_app_cuisine/src/data/data_recipes_tokenised.json does not exist

In [ ]:
df_loaded.iloc[0]

name                        arriba   baked winter squash mexican style
id                                                              137739
minutes                                                             55
contributor_id                                                   47892
submitted                                          2005-09-16 00:00:00
tags                 ['60-minutes-or-less', 'time-to-make', 'course...
nutrition                        [51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]
n_steps                                                             11
steps                ['make a choice and proceed with recipe', 'dep...
description          autumn is my favorite time of year to cook! th...
ingredients          [squash, seasoning, spice, honey, butter, oil,...
n_ingredients                                                        7
i                                                               145702
name_tokens          [40480, 1943, 6987, 12187, 4423, 23841, 12432,...
ingred

In [ ]:
df_loaded == df_recipes_tokenised

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178260,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
178261,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
178262,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
178263,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [ ]:
matcher_1 = IngredientMatcher(df_loaded,dico_all_month_ingredient)
matcher_1.seasonal_recommendations_1('strawberry',5)

[398,
 14662,
 17028,
 17734,
 46449,
 65664,
 67075,
 68157,
 68522,
 89267,
 99999,
 100888,
 103134,
 119242,
 133399,
 133437,
 154588,
 154675,
 154686,
 154713,
 154759,
 154803,
 154913,
 155035,
 155047,
 155066,
 155068,
 155091,
 155205,
 155210,
 155232,
 155332,
 156771,
 172681]

In [ ]:
matcher_1.ingredient_match('strawberry', 5)

Counter({'strawberry': 588,
         'sugar': 417,
         'juice': 199,
         'cream': 155,
         'water': 136,
         'salt': 119,
         'butter': 112,
         'cheese': 110,
         'flour': 91,
         'milk': 90,
         'extract': 88,
         'egg': 88,
         'oil': 63,
         'banana': 60,
         'powder': 55,
         'yogurt': 55,
         'blueberry': 47,
         'cornstarch': 47,
         'cinnamon': 45,
         'vinegar': 45,
         'honey': 43,
         'vanilla': 39,
         'pepper': 37,
         'soda': 37,
         'mix': 35,
         'gelatin': 31,
         'orange': 31,
         'syrup': 31,
         'onion': 27,
         'of': 26,
         'cube': 26,
         'leaf': 25,
         'liqueur': 25,
         'whip': 25,
         'raspberry': 25,
         'pineapple': 24,
         'almond': 23,
         'topping': 23,
         'fruit': 22,
         'rhubarb': 22,
         'grape': 22,
         'white': 21,
         'concentrate': 21,
        